In [115]:
import pandas as pd
import numpy as np
import folium
import plotly.express as px
import plotly.graph_objects as go
import datetime
from datetime import datetime
import requests
from dateutil.parser import parse

In [116]:
df = pd.read_csv('Locatievoorstellen.csv', header=1, index_col=0)

In [117]:
pd.set_option('display.max_columns', None)

In [118]:
df.head()

,Locatievoorstel ID,Stilgezet,Stilgezet uitleg,Stilgezet nu of ooit in verleden,Tijdstip van laatste wijziging,Verplaatsing MRA-E Laadpaal ID,Tijdstip aangemaakt,Tijdstip locatievoorstel,Tijdstip controle locatievoorstel,Tijdstip advies exploitant,Tijdstip publicatie verkeersbesluit,Tijdstip aanvraag netaansluiting,Tijdstip definitief verkeersbesluit,Tijdstip accepteren MRA-E,Tijdstip realisatie plannen,Tijdstip locatie voorbereiden,Tijdstip in bedrijf,Tijdstip opleveren laadpaal,Tijdstip opleveren locatie,Organisatie ID,Organisatie naam,Gemeente ID,Gemeente naam,Referenties,Locatie - Longitude,Locatie - Latitude,Straat,Huisnummer,Postcode,Plaats,Was voorkeurslocatie,VoorkeurslocatieID,Opmerkingen,Autodelen,Laadplein,Laadpunten,Gereserveerde parkeervakken,Gereserveerde parkeervakken autodelen,Aanleiding,Toelichting bij verplaatsing,Locatiecode,Aanbodgestuurd Opmerkingen,Datagestuurd Opmerkingen,Exploitant Akkoord,Aannemer Akkoord,Gemeente Akkoord,Gemeente akkoord met offerte verplaatsing,Verkeersbesluit Gepubliceerd Datum,Verkeersbesluit Definitief Datum,Verkeersbesluit Definitief Gewijzigd,Aanvraag Netaansluiting Datum,Datum Realisatie Gepland,Datum In Bedrijf,Laadpaal ID,Extra Laadpaal IDs,Datum oplevering
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,LV-6,False,NaN,False,2020-10-22 09:06:27.400000,NaN,2018-12-05 13:40:44.360000,2018-12-05 14:06:07.114000,2018-12-19 14:45:00.395000,2018-12-24 14:47:22.368000,2018-12-31 08:50:02.188000,2019-01-31 13:11:05.446000,2018-12-31 10:46:40.870000,2018-12-31 10:46:44.903000,2019-04-24 05:54:56.583000,2019-04-24 06:36:17.898000,2019-05-08 08:36:20.072000,2019-08-14 08:56:36.783000,2019-09-11 07:24:17.680000,84,OVER-Gemeenten,385,Wormerland,"WORMERLAND00003, WORMERLAND00009",4.810485,52.494985,Spatterstraat,21,1531 DA,Wormer,False,<nil>,bij sporthal,False,False,2,2,0,vraaggestuurdverzoek,NaN,NaN,NaN,NaN,True,True,NaN,NaN,2018-08-28 00:00:00,2018-10-09 00:00:00,False,2019-01-21 00:00:00,2019-05-06 00:00:00,2019-05-07 00:00:00,PP001911,NaN,2019-05-08 00:00:00
2,LV-8,False,NaN,False,2020-10-22 09:06:27.400000,NaN,2018-12-06 09:20:44.736000,2018-12-31 12:39:10.967000,2018-12-31 12:39:30.972000,2019-01-02 08:06:41.882000,2019-02-25 15:19:54.254000,2019-03-07 06:43:14.800000,2019-04-09 07:55:49.334000,2019-04-09 08:45:33.427000,2019-09-11 06:55:01.214000,2019-09-25 12:08:00.393000,2019-09-25 08:24:13.355000,2019-10-17 14:27:06.405000,2019-09-25 12:08:18.781000,16,Gemeente De Bilt,269,De Bilt,"DEBILT00016, DEBILT00025",5.180691,52.111028,Buys Ballotweg,66,3731 VK,De Bilt,False,<nil>,NaN,False,False,2,2,0,vraaggestuurdverzoek,NaN,NaN,NaN,NaN,True,True,NaN,NaN,2019-02-27 00:00:00,2019-04-10 00:00:00,False,2019-03-06 00:00:00,2019-09-25 00:00:00,2019-09-25 00:00:00,PP001968,NaN,2019-09-25 00:00:00
3,LV-11,False,NaN,False,2020-10-22 09:06:27.400000,NaN,2018-12-07 09:59:17.463000,2018-12-07 10:03:25.510000,2019-01-14 12:10:51.520000,2019-01-15 08:45:00.559000,2019-01-31 15:28:01.963000,2019-02-18 14:02:10.212000,2019-02-20 07:31:18.342000,2019-02-20 11:24:01.657000,2019-06-07 06:57:55.205000,2019-06-11 06:57:01.107000,2019-06-13 12:08:19.299000,2019-08-14 08:58:25.607000,2020-07-16 14:50:16.003000,28,Gemeente Heemskerk,334,Heemskerk,HEEMSKERK00007,4.655286,52.511481,Commandeurslaan,81,1964 LW,Heemskerk,False,<nil>,NaN,False,False,2,2,0,vraaggestuurdverzoek,NaN,NaN,NaN,NaN,True,True,NaN,NaN,2019-01-25 00:00:00,2019-02-27 00:00:00,False,2019-02-14 00:00:00,2019-06-11 00:00:00,2019-06-13 00:00:00,PP002024,NaN,2019-06-13 00:00:00
4,LV-12,False,NaN,False,2020-10-22 09:06:27.400000,NaN,2018-12-07 10:17:23.991000,2018-12-07 10:18:44.540000,2018-12-19 14:45:58.376000,2018-12-24 14:49:50.718000,2019-01-31 15:33:03.486000,2019-02-18 14:04:13.754000,2019-02-20 07:25:32.040000,2019-02-20 11:24:13.567000,2019-06-07 07:00:04.976000,2019-06-11 06:59:11.801000,2019-06-13 12:07:04.829000,2019-08-14 09:00:13.784000,2020-07-16 14:55:31.495000,28,Gemeente Heemskerk,334,Heemskerk,HEEMSKERK00008,4.670446,5

In [119]:
df.columns = df.columns.str.replace(' - ', '_')
df.columns = df.columns.str.replace(' ', '_')
df.columns = df.columns.str.lower()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5633 entries, 1 to 5633
Data columns (total 56 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   locatievoorstel_id                         5633 non-null   object 
 1   stilgezet                                  5633 non-null   bool   
 2   stilgezet_uitleg                           394 non-null    object 
 3   stilgezet_nu_of_ooit_in_verleden           5633 non-null   bool   
 4   tijdstip_van_laatste_wijziging             5633 non-null   object 
 5   verplaatsing_mra-e_laadpaal_id             78 non-null     float64
 6   tijdstip_aangemaakt                        5633 non-null   object 
 7   tijdstip_locatievoorstel                   5603 non-null   object 
 8   tijdstip_controle_locatievoorstel          5591 non-null   object 
 9   tijdstip_advies_exploitant                 5582 non-null   object 
 10  tijdstip_publicatie_verk

In [120]:
df["tijdstip_van_laatste_wijziging"] = pd.to_datetime(df["tijdstip_van_laatste_wijziging"])
df["tijdstip_aangemaakt"] = pd.to_datetime(df["tijdstip_aangemaakt"])
df["tijdstip_locatievoorstel"] = pd.to_datetime(df["tijdstip_locatievoorstel"])
df["tijdstip_controle_locatievoorstel"] = pd.to_datetime(df["tijdstip_controle_locatievoorstel"])
df["tijdstip_advies_exploitant"] = pd.to_datetime(df["tijdstip_advies_exploitant"])
df["tijdstip_publicatie_verkeersbesluit"] = pd.to_datetime(df["tijdstip_publicatie_verkeersbesluit"])
df["tijdstip_aanvraag_netaansluiting"] = pd.to_datetime(df["tijdstip_aanvraag_netaansluiting"])
df["tijdstip_definitief_verkeersbesluit"] = pd.to_datetime(df["tijdstip_definitief_verkeersbesluit"])
df["tijdstip_accepteren_mra-e"] = pd.to_datetime(df["tijdstip_accepteren_mra-e"])
df["tijdstip_realisatie_plannen"] = pd.to_datetime(df["tijdstip_realisatie_plannen"])
df["tijdstip_locatie_voorbereiden"] = pd.to_datetime(df["tijdstip_locatie_voorbereiden"])
df["tijdstip_in_bedrijf"] = pd.to_datetime(df["tijdstip_in_bedrijf"])
df["tijdstip_opleveren_laadpaal"] = pd.to_datetime(df["tijdstip_opleveren_laadpaal"])
df["tijdstip_opleveren_locatie"] = pd.to_datetime(df["tijdstip_opleveren_locatie"])
df['datum_oplevering'] = pd.to_datetime(df['datum_oplevering'])

In [121]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5633 entries, 1 to 5633
Data columns (total 56 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   locatievoorstel_id                         5633 non-null   object        
 1   stilgezet                                  5633 non-null   bool          
 2   stilgezet_uitleg                           394 non-null    object        
 3   stilgezet_nu_of_ooit_in_verleden           5633 non-null   bool          
 4   tijdstip_van_laatste_wijziging             5633 non-null   datetime64[ns]
 5   verplaatsing_mra-e_laadpaal_id             78 non-null     float64       
 6   tijdstip_aangemaakt                        5633 non-null   datetime64[ns]
 7   tijdstip_locatievoorstel                   5603 non-null   datetime64[ns]
 8   tijdstip_controle_locatievoorstel          5591 non-null   datetime64[ns]
 9   tijdstip_advies_exp

In [122]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5633 entries, 1 to 5633
Data columns (total 56 columns):
 #   Column                                     Non-Null Count  Dtype         
---  ------                                     --------------  -----         
 0   locatievoorstel_id                         5633 non-null   object        
 1   stilgezet                                  5633 non-null   bool          
 2   stilgezet_uitleg                           394 non-null    object        
 3   stilgezet_nu_of_ooit_in_verleden           5633 non-null   bool          
 4   tijdstip_van_laatste_wijziging             5633 non-null   datetime64[ns]
 5   verplaatsing_mra-e_laadpaal_id             78 non-null     float64       
 6   tijdstip_aangemaakt                        5633 non-null   datetime64[ns]
 7   tijdstip_locatievoorstel                   5603 non-null   datetime64[ns]
 8   tijdstip_controle_locatievoorstel          5591 non-null   datetime64[ns]
 9   tijdstip_advies_exp

In [123]:
df.head()

,locatievoorstel_id,stilgezet,stilgezet_uitleg,stilgezet_nu_of_ooit_in_verleden,tijdstip_van_laatste_wijziging,verplaatsing_mra-e_laadpaal_id,tijdstip_aangemaakt,tijdstip_locatievoorstel,tijdstip_controle_locatievoorstel,tijdstip_advies_exploitant,tijdstip_publicatie_verkeersbesluit,tijdstip_aanvraag_netaansluiting,tijdstip_definitief_verkeersbesluit,tijdstip_accepteren_mra-e,tijdstip_realisatie_plannen,tijdstip_locatie_voorbereiden,tijdstip_in_bedrijf,tijdstip_opleveren_laadpaal,tijdstip_opleveren_locatie,organisatie_id,organisatie_naam,gemeente_id,gemeente_naam,referenties,locatie_longitude,locatie_latitude,straat,huisnummer,postcode,plaats,was_voorkeurslocatie,voorkeurslocatieid,opmerkingen,autodelen,laadplein,laadpunten,gereserveerde_parkeervakken,gereserveerde_parkeervakken_autodelen,aanleiding,toelichting_bij_verplaatsing,locatiecode,aanbodgestuurd_opmerkingen,datagestuurd_opmerkingen,exploitant_akkoord,aannemer_akkoord,gemeente_akkoord,gemeente_akkoord_met_offerte_verplaatsing,verkeersbesluit_gepubliceerd_datum,verkeersbesluit_definitief_datum,verkeersbesluit_definitief_gewijzigd,aanvraag_netaansluiting_datum,datum_realisatie_gepland,datum_in_bedrijf,laadpaal_id,extra_laadpaal_ids,datum_oplevering
0,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,LV-6,False,NaN,False,2020-10-22 09:06:27.400,NaN,2018-12-05 13:40:44.360,2018-12-05 14:06:07.114,2018-12-19 14:45:00.395,2018-12-24 14:47:22.368,2018-12-31 08:50:02.188,2019-01-31 13:11:05.446,2018-12-31 10:46:40.870,2018-12-31 10:46:44.903,2019-04-24 05:54:56.583,2019-04-24 06:36:17.898,2019-05-08 08:36:20.072,2019-08-14 08:56:36.783,2019-09-11 07:24:17.680,84,OVER-Gemeenten,385,Wormerland,"WORMERLAND00003, WORMERLAND00009",4.810485,52.494985,Spatterstraat,21,1531 DA,Wormer,False,<nil>,bij sporthal,False,False,2,2,0,vraaggestuurdverzoek,NaN,NaN,NaN,NaN,True,True,NaN,NaN,2018-08-28 00:00:00,2018-10-09 00:00:00,False,2019-01-21 00:00:00,2019-05-06 00:00:00,2019-05-07 00:00:00,PP001911,NaN,2019-05-08
2,LV-8,False,NaN,False,2020-10-22 09:06:27.400,NaN,2018-12-06 09:20:44.736,2018-12-31 12:39:10.967,2018-12-31 12:39:30.972,2019-01-02 08:06:41.882,2019-02-25 15:19:54.254,2019-03-07 06:43:14.800,2019-04-09 07:55:49.334,2019-04-09 08:45:33.427,2019-09-11 06:55:01.214,2019-09-25 12:08:00.393,2019-09-25 08:24:13.355,2019-10-17 14:27:06.405,2019-09-25 12:08:18.781,16,Gemeente De Bilt,269,De Bilt,"DEBILT00016, DEBILT00025",5.180691,52.111028,Buys Ballotweg,66,3731 VK,De Bilt,False,<nil>,NaN,False,False,2,2,0,vraaggestuurdverzoek,NaN,NaN,NaN,NaN,True,True,NaN,NaN,2019-02-27 00:00:00,2019-04-10 00:00:00,False,2019-03-06 00:00:00,2019-09-25 00:00:00,2019-09-25 00:00:00,PP001968,NaN,2019-09-25
3,LV-11,False,NaN,False,2020-10-22 09:06:27.400,NaN,2018-12-07 09:59:17.463,2018-12-07 10:03:25.510,2019-01-14 12:10:51.520,2019-01-15 08:45:00.559,2019-01-31 15:28:01.963,2019-02-18 14:02:10.212,2019-02-20 07:31:18.342,2019-02-20 11:24:01.657,2019-06-07 06:57:55.205,2019-06-11 06:57:01.107,2019-06-13 12:08:19.299,2019-08-14 08:58:25.607,2020-07-16 14:50:16.003,28,Gemeente Heemskerk,334,Heemskerk,HEEMSKERK00007,4.655286,52.511481,Commandeurslaan,81,1964 LW,Heemskerk,False,<nil>,NaN,False,False,2,2,0,vraaggestuurdverzoek,NaN,NaN,NaN,NaN,True,True,NaN,NaN,2019-01-25 00:00:00,2019-02-27 00:00:00,False,2019-02-14 00:00:00,2019-06-11 00:00:00,2019-06-13 00:00:00,PP002024,NaN,2019-06-13
4,LV-12,False,NaN,False,2020-10-22 09:06:27.400,NaN,2018-12-07 10:17:23.991,2018-12-07 10:18:44.540,2018-12-19 14:45:58.376,2018-12-24 14:49:50.718,2019-01-31 15:33:03.486,2019-02-18 14:04:13.754,2019-02-20 07:25:32.040,2019-02-20 11:24:13.567,2019-06-07 07:00:04.976,2019-06-11 06:59:11.801,2019-06-13 12:07:04.829,2019-08-14 09:00:13.784,2020-07-16 14:55:31.495,28,Gemeente Heemskerk,334,Heemskerk,HEEMSKERK00008,4.670446,52.508710,Maerten van Heemskerckstraat,25,1961 EB,Heemskerk,False,<nil>,NaN,False,False,2,2,0,vraaggestuurdverzoek,NaN,NaN,NaN,NaN,True,True,NaN,NaN,2019-01-18 00:00:00,2019-02-20 00:00:00,False,2019-

In [124]:
df.isna().sum()

locatievoorstel_id                              0
stilgezet                                       0
stilgezet_uitleg                             5239
stilgezet_nu_of_ooit_in_verleden                0
tijdstip_van_laatste_wijziging                  0
verplaatsing_mra-e_laadpaal_id               5555
tijdstip_aangemaakt                             0
tijdstip_locatievoorstel                       30
tijdstip_controle_locatievoorstel              42
tijdstip_advies_exploitant                     51
tijdstip_publicatie_verkeersbesluit           572
tijdstip_aanvraag_netaansluiting              619
tijdstip_definitief_verkeersbesluit           762
tijdstip_accepteren_mra-e                     788
tijdstip_realisatie_plannen                  1705
tijdstip_locatie_voorbereiden                1917
tijdstip_in_bedrijf                          1861
tijdstip_opleveren_laadpaal                  1912
tijdstip_opleveren_locatie                   1995
organisatie_id                                  0


In [125]:
df = df.drop(columns=['stilgezet_uitleg','verplaatsing_mra-e_laadpaal_id', 'opmerkingen', 'toelichting_bij_verplaatsing', 
                      'locatiecode', 'aanbodgestuurd_opmerkingen', 'datagestuurd_opmerkingen', 'gemeente_akkoord_met_offerte_verplaatsing',
                     'extra_laadpaal_ids', 'referenties', 'aanleiding'])

In [126]:
df['doorlooptijd_d'] = df['tijdstip_opleveren_locatie'] - df['tijdstip_aangemaakt']

In [127]:
df = df.loc[(df['doorlooptijd_d'].notna())]

In [128]:
df['postcode'] = df['postcode'].str.replace('[a-zA-Z]', '', regex=True)
df['postcode'] = df['postcode'].str.replace(' ', '', regex=True)

In [129]:
df['datum_oplevering_y'] = pd.DatetimeIndex(df['datum_oplevering']).year

In [130]:
df = df.loc[(df['datum_oplevering_y'] < 2024)]

In [131]:
fig = px.histogram(df, x='datum_oplevering_y')

fig.show()